<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="float:right; max-width: 250px; display: inline"  alt="Wikistat"/></a>

</center>

# [Scénarios d'Apprentissage Statistique](https://github.com/wikistat/Apprentissage)



# Gestion des données manquantes sur les données d'Ozone avec  <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a>  <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="Scikit-learn"/></a>

**Résumé**: 
- Création d'un jeu de données contenant des données manquantes à partir des données ozone 
- Visualisation des données manquantes 
- Comparaison de diverses méthodes de complétion pour les données quantitatives
- Complétion avec MissForest de l'ensemble des données (quantitaives et qualitatives) et impact sur les résultats de classification relativement au jeu de données initial complet

## Prise en charge des données </font>
### Lesture des données

Les données ont été extraites et mises en forme par le service concerné de Météo France. Elles sont décrites par les variables suivantes:

* **JOUR** Le type de jour ; férié (1) ou pas (0) ;
* **O3obs** La concentration d'ozone effectivement observée le lendemain à 17h locales correspondant souvent au maximum de pollution observée ;
* **MOCAGE** Prévision de cette pollution obtenue par un modèle déterministe de mécanique des fluides (équation de Navier et Stockes);
* **TEMPE** Température prévue par MétéoFrance pour le lendemain 17h ;
* **RMH2O** Rapport d'humidité ;
* **NO2** Concentration en dioxyde d'azote ;
* **NO** Concentration en monoxyde d'azote ;
* **STATION** Lieu de l'observation : Aix-en-Provence, Rambouillet, Munchhausen, Cadarache et Plan de Cuques ;
* **VentMOD** Force du vent ;
* **VentANG** Orientation du vent. 

Ce sont des données "propres", sans trous, bien codées et de petites tailles. Elles présentent avant tout un caractère pédagogique.

Il est choisi ici de lire les données avec la librairie `pandas` pour bénéficier de la classe DataFrame. Ce n'est pas nécessaire pour l'objectif de prévision car les variables qualitatives ainsi construites ne peuvent être utilisées pour l'interprétation des modèles obtenus dans `scikit-learn` qui ne reconnaît pas la classe DataFrame.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Lecture des données
## Charger les données ou les lire directement en précisant le chemin
path=""
ozone=pd.read_csv(path+"depSeuil.dat",sep=",",header=0)
# Vérification du contenu
ozone.head()

Ce qui suit permet d'affecter le bon type aux variables.

In [ ]:
ozone["STATION"]=pd.Categorical(ozone["STATION"],ordered=False)
ozone["JOUR"]=pd.Categorical(ozone["JOUR"],ordered=False)
ozone["O3obs"]=pd.DataFrame(ozone["O3obs"], dtype=float)
ozone.dtypes

In [ ]:
ozone.describe()

### Transformations des données

In [ ]:
from math import sqrt, log
ozone["SRMH2O"]=ozone["RMH2O"].map(lambda x: sqrt(x))
ozone["LNO2"]=ozone["NO2"].map(lambda x: log(x))
ozone["LNO"]=ozone["NO"].map(lambda x: log(x))
del ozone["RMH2O"]
del ozone["NO2"]
del ozone["NO"]


In [ ]:
ozone["DepSeuil"]=ozone["O3obs"].map(lambda x: x > 150)
ozone.head()

In [ ]:
# variable à expliquer binaire
Yb=ozone["DepSeuil"].map(lambda x: int(x))


### Séparation des variables quantitatives et qualitatives

In [ ]:
ozone.head()

In [ ]:
# Variables explicatives
# On transforme les variables qualitatives en paquets d'indicatrice pour la phase d'apprentissage.

ozoneDum=pd.get_dummies(ozone[["JOUR","STATION"]])
del ozoneDum["JOUR_0"]
ozoneQuant=ozone[["MOCAGE","TEMPE","VentMOD","VentANG","SRMH2O","LNO2","LNO"]]

dfC=pd.concat([ozoneDum,ozoneQuant],axis=1)
dfC.head()

In [ ]:
ozoneQuant.head()

## [Gestion des données manquantes](http://wikistat.fr/pdf/st-m-app-idm.pdf)
Les vraies données sont le plus souvent mitées par l'absence de données, conséquences d'erreurs de saisie, de pannes de capteurs... Les librairies de Python (`pandas`) offrent des choix rudimentaires pour faire des imputations de données manquantes quand celles-ci le sont de façon complètement aléatoire. 

Le calepin R d'analyse de ces mêmes données propose une comparaison assez détaillée de deux stratégiées afin d'évaluer leurs performances respectives. 

La **première stratégie** commence par imputer les données manquantes en les prévoyant par l'algorithme `missForest`. Une fois les données manquantes imputées, on utilise les forêts aléatoires pour construire un algorithme de prédiction du dépassement du seuil.

La **deuxième stratégie** évite l'étape d'imputation en exécutant directement un algorithme de prévision tolérant des données manquantes. Peu le font, c'est le cas de `XGBoost`.

Sur ces données, mais sans gros effort d'optimisation de `XGBoost`, la première stratégie enchaînant `missForest` puis `randomForest` conduit à de meilleurs résultats. Seule celle-ci est employée dans ce calepin mais, bien évidemment, l'exécution de `xgboost` sans imputation préalable est une option également possible en Python.

Bien moins de méthodes sont proposées en Python, `Scikit-learn` ne proposant que des imputations basiques par la moyenne ou la médiane comme dans `pandas`. Néanmoins une imputation par prévision utilisant *k*-nn,  ou des forêts aléatoires (Missforest) est disponible dans la librairie `sklearn.impute`. Le souci est que Python ne gère pas bien les deux types de variables : quantitatives et qualitatives. Pour simplifier dans ce TP, nous allons donc ne considérer des données manquantes que dans les variables quantitatives. 

Les commandes ci-dessous font appel aux fichiers suivants:
- `X` données complètes initiales : **ozoneQuant**
- `Xna` les données avec des trous, 
- `XnaImp` les données avec imputations 


### Préparation des trous dans `ozone`
Les variables explicatives quantitatives de la base `ozone` sont reprises. La première opération consiste à générer aléatoirement un certain taux de données manquantes par la fonction définie ci-dessous.

In [ ]:
import numpy as np
import numpy.ma as ma
import random

def input_nan(x, tx):
    """
    x : a 2D matrix of float dtype
    tx: the rate of nan value to put in the matrix
    """
    n_total = x.shape[0] * x.shape[1]
    mask = np.array([random.random() for _ in range(n_total)]).reshape(x.shape)<tx
    mx = ma.masked_array(x, mask=mask, fill_value=np.nan)
    return mx.filled()

In [ ]:
# données initiales 
X=ozoneQuant 
# Génération de 30% de valeurs manquantes
Xna=input_nan(X, .3)

Xna_df = pd.DataFrame(Xna, columns=ozoneQuant.columns)

Xna_df.head()

### Visualisation des données manquantes

In [ ]:
nrows = len(Xna)
missing_rates = 1-Xna_df.count(axis=0)/nrows
missing_rates

In [ ]:
missing_rates.plot.bar()

## Imputation  des données manquantes

### Imputation simple

In [ ]:
from sklearn.impute import SimpleImputer

X_mean = SimpleImputer().fit_transform(Xna)


**Question** Regarder quelles sont les options proposées par SimpleImputer. Quelle est l'option par défaut utilisée ci dessus ?

In [ ]:
X_meanImp=pd.DataFrame(X_mean, columns=ozoneQuant.columns)

In [ ]:
X_meanImp.head()

**Question** Sur la variable 'Température', reprendre l'analyse qui avait été faite en R : boxplot des erreurs d'imputation avec les diverses méthodes (moyenne, médiane, ..)

### Imputation avec KNN

In [ ]:
from sklearn.impute import  KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
X_kNN = knn_imputer.fit_transform(Xna)

In [ ]:
X_kNNImp=pd.DataFrame(X_kNN, columns=ozoneQuant.columns)

In [ ]:
X_kNNImp.head()

### Imputation par `missForest`

L'estimateur *ExtraTreesRegressor* entraîne une forêt aléatoire itérative et imite *missForest* dans R. *ExtraTreesRegressor* ajuste un certain nombre d'arbres aléatoires  et calcule la moyenne des résultats. Il provient du module sklearn.ensemble. Ses principaux arguments sont le nombre d'arbres dans la forêt et l'état aléatoire qui permet de contrôler les sources d'aléa.  Regarder l'aide pour plus de précisions.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
estimator_rf = ExtraTreesRegressor(n_estimators=20, random_state=0)
X_rf = IterativeImputer(estimator=estimator_rf, random_state=0, max_iter=300).fit_transform(Xna)


In [ ]:
X_rfImp=pd.DataFrame(X_rf, columns=ozoneQuant.columns)

In [ ]:
X_rfImp.head()

In [ ]:
ozoneQuant.head()

## Comparaison des erreurs de prévision par forêt aléatoire
Prévision du dépassement d'ozone sans données manquantes et avec données manquantes imputées. Comparaison des erreurs de prévision sur l'échantillon test. Les valeurs par défaut des paramètres sont conservées. 
### Prévision sans données manquantes

Extractions des échantillons d'apprentissage  et test. Comme le générateur est initalisé de façon identique, ce sont les mêmes échantillons dans les deux cas.

In [ ]:
from sklearn.model_selection import train_test_split  
X_train,X_test,Yb_train,Yb_test=train_test_split(dfC,Yb,test_size=200,random_state=11)


In [ ]:
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()  
scaler.fit(X_train)  
Xr_train = scaler.transform(X_train)  
# Meme transformation sur le test
Xr_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
# prévision sans trous
forest = RandomForestClassifier(n_estimators=500)
# apprentissage
rfFit = forest.fit(Xr_train,Yb_train)
# erreur de prévision
# erreur de prévision sur le test
1-rfFit.score(Xr_test,Yb_test)

### Prévision après imputation des données manquantes

In [ ]:
dfCImp=pd.concat([ozoneDum,X_rfImp],axis=1)
dfCImp.head()

In [ ]:
from sklearn.model_selection import train_test_split  
XnaImp_train,XnaImp_test,Yb_train,Yb_test=train_test_split(dfCImp,Yb,test_size=200,random_state=11)


In [ ]:
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()  
scaler.fit(X_train)  
Xr_train = scaler.transform(XnaImp_train)  
# Meme transformation sur le test
Xr_test = scaler.transform(XnaImp_test)

In [ ]:
# prévision avec trous imputés
forest = RandomForestClassifier(n_estimators=500)
# apprentissage
rfFit = forest.fit(XnaImp_train,Yb_train)
# erreur de prévision
# erreur de prévision sur le test
1-rfFit.score(XnaImp_test,Yb_test)

**Question** Que dire de la qualité de prévision avec 30% de données manquantes ? Comparer avec ce qui est obtenu pour les autres types d'imputation.

Faites varier ce taux et étudiez la dégradation de la prévision.


